# Finne alle emne dette semesteret
Eg leiter opp alle emne dette semesteret, og sjekke rom det er nokon emne med lærar og student ssom ikkje er publisert.

In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


Her hentar eg inn alle emna, og sjekker dei mot det aktuelle semesteret. Det er alle emne som har undervising før, samtidig med, eller etter "2025 HØST".

In [2]:
url = "https://hvl.instructure.com/api/v1/accounts/1/courses"
terminar = pd.read_csv("terminar.csv").to_numpy()


In [3]:
dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        # print(respons.headers)
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            if element['enrollment_term_id'] in terminar:
                dataliste.append([element['id'], element['workflow_state']])
        df = pd.DataFrame(dataliste, columns=['id', 'workflow_state'])
        dr_liste.append(df)
    # hentmeir = False
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

https://hvl.instructure.com/api/v1/accounts/1/courses?page=2&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=3&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=4&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=5&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=6&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=7&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=8&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=9&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=10&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=11&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=12&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=13&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/courses?page=14&per_page=100
https://hvl.instructure.com/api/v

No vil ramma *alledata* innehalde Canvas-ID for alle emne. Så kan eg gå gjennom kvart emne og hente ut litt info:

- er det nokon med "enrollment" lik "teacher"?
- status for emnet

Eg lagrer fila slik at eg slepp å hente den fram att ein gang til.

In [6]:
alledata.to_csv('alledata.csv', index=False)  # Lagre til CSV-fil for vidare bruk